In [5]:
import math
from collections import Counter
def Convert_to_LargeNumber(asci):
        R = 2675
        prod = 1
        for  e in asci:
                prod*=(R+2*e)
        return(prod//2) 

    
def HashFun(k):
    #A = (math.sqrt(5)-1)/2
        
    #use Bash command to get the number of passwords in the file using "wc -l password2.txt" and they are 110000000
    #m = 2*110000000 # we create 10X slots for the 
    #w = 2** 256 #
    #A = 2654435769 / w 
    #return(math.floor(m*(k*A - int(k*A)))) # As stated in the book we use floor(m(kAmod1))
    w = 4294967295 # 2^32
    A = 2654435769
    r0 = k * A
    p = 32
    return ( r0 & w ) >> ( 32 - p )

passwords = open('passwords2.txt','r')
#row = 1
f = open('HashValue2.txt','w+')
#Recommend to use the if condition in the for loop for testing
for p in passwords:
    #if(row==10): #100000000 we generated a data for this row
     #   break
    list = [ord(char) for char in p.strip()]
    #print("Key------: " + p.strip() + "  LargeNumber-----: " + str(Convert_to_LargeNumber(list)) + " HashValue--: " + str(HashFun(Convert_to_LargeNumber(list))))
    
    #If We want to only store the hash value into a file we remove the p.stripe() and str(Convert_to_LargeNumber(list))
    #f.write(p.strip()+" "+str(Convert_to_LargeNumber(list))+" "+str(HashFun(Convert_to_LargeNumber(list)))+"\n")
    #hashvalue.append(HashFun(Convert_to_LargeNumber(list)))
    #row+=1
    f.write(str(HashFun(Convert_to_LargeNumber(list)))+"\n")

passwords.close()
f.close()

# Read the HashValue using pyspark

In [6]:
HashValue = sc.textFile('HashValue2.txt') 

We use the Hash value as a word, and we count the number of occurences of each Hash Value

In [7]:
OccurenceCount = HashValue.flatMap(lambda line: line.split(",")).map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

To identify the duplicates we filter the hash values that are counted to be more than 1.

In [8]:
Duplicates = OccurenceCount.filter(lambda x: x[1]>1).count()

In [9]:
Duplicates

10916162

## AABA != "AAAB

In [28]:
import math
from collections import Counter
def Convert_to_LargeNumber(ascii):
        R = 265
        a = []
        for  e in ascii:
                a.append(e)
        return (''.join(map(str,a))) # The two strings are indentical if their ascii are the same sequencially if not they are different

    
def HashFun(k):
     #A = (math.sqrt(5)-1)/2
        
    w = 4294967295 # 4294967296 # 2^32
    A = 2654435769
    r0 = k * A
    p = 32
    return ( r0 & w ) >> ( 32 - p ) # As stated in the book we use floor(m(kAmod1))

passwords = open('passwords2.txt','r')
#row = 1
f = open('HashValue5.txt','w+')
#Recommend to use the if condition in the for loop for testing
for p in passwords:
    #if(row==10): #100000000 we generated a data for this row
     #   break
    list = [ord(char) for char in p.strip()]
    #print("Key------: " + p.strip() + "  HashValue-----: " + str(HashFun(int(Convert_to_LargeNumber(list)))))#str(Convert_to_LargeNumber(list)))# + " HashValue--: " + str(HashFun(Convert_to_LargeNumber(list))))
    #If We want to only store the hash value into a file we remove the p.stripe() and str(Convert_to_LargeNumber(list))
    #f.write(p.strip()+" "+str(Convert_to_LargeNumber(list))+" "+str(HashFun(Convert_to_LargeNumber(list)))+"\n")
    #hashvalue.append(HashFun(Convert_to_LargeNumber(list)))
    #row+=1
    f.write(p.strip() +" "+str(HashFun(int(Convert_to_LargeNumber(list))))+"\n")

passwords.close()
f.close()

# We repeat the above code to identify the duplicates

In [ ]:
HashValue1 = sc.textFile('HashValue4.txt') 

In [ ]:
OccurenceCount = HashValue1.flatMap(lambda line: line.split(",")).map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [ ]:
Duplicates1 = OccurenceCount.filter(lambda x: x[1]>1).count()